# Week 6: Development of a midlatitude cyclone through baroclinic instability
*MAQ - 32806, Chiel van Heerwaarden & Imme Benedict, 2018*

In this assignment you will investigate the time evolution of a midlatitude cyclone over the Atlantic Ocean. For this, you will use ECMWF ERA5 data from 3 to 7 January 2014. The data has steps of 6 h and 4 pressure levels are contained: 850, 500, 300, and 200 hPa.
___

# Setting up Python and loading the data
We start the tutorial by loading the required Python packages and setting the figure properties.

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install netCDF4

# Loading the packages.
import numpy as np              # Numpy is the fundamental package for scientific computing in Python.
import netCDF4 as nc            # NetCDF is the data format of the meteorological data that we use.
import matplotlib.pyplot as plt # Matplotlib is a scientific plotting package.
import datetime                 # Datetime is a package to deal with dates.
import cartopy.crs as ccrs
import cartopy
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from matplotlib.colors import LinearSegmentedColormap
from ipywidgets import interact

# The statement below enforces the plots to be put into this notebook, instead of in their own windows.
%matplotlib inline

In [ ]:
plt.rcParams.update({'font.size': 11})          # Set the standard font size of the plots to 11pt.
plt.rcParams.update({'figure.figsize': [13,5]}) # Set the standard figure size.

In [ ]:
# Create custom color map similar to the NCAR NCL WhiteBlueGreenYellowRed
cdict = {'red':   ((   0/253., 255./255., 255./255.),
                   (  36/253., 157./255., 157./255.),
                   (  72/253.,  72./255.,  72./255.),
                   ( 108/253.,  73./255.,  73./255.),
                   ( 145/253., 250./255., 250./255.),
                   ( 181/253., 245./255., 245./255.),
                   ( 217/253., 211./255., 211./255.),
                   ( 253/253., 146./255., 146./255.)),
         'green': ((   0/253., 255./255., 255./255.),
                   (  36/253., 218./255., 218./255.),
                   (  72/253., 142./255., 142./255.),
                   ( 108/253., 181./255., 181./255.),
                   ( 145/253., 232./255., 232./255.),
                   ( 181/253., 106./255., 106./255.),
                   ( 217/253.,  31./255.,  31./255.),
                   ( 253/253.,  21./255.,  21./255.)),
         'blue':  ((   0/253., 255./255., 255./255.),
                   (  36/253., 247./255., 247./255.),
                   (  72/253., 202./255., 202./255.),
                   ( 108/253.,  70./255.,  70./255.),
                   ( 145/253.,  92./255.,  92./255.),
                   ( 181/253.,  45./255.,  45./255.),
                   ( 217/253.,  40./255.,  40./255.),
                   ( 253/253.,  25./255.,  25./255.))}

my_cmap = LinearSegmentedColormap('my_colormap', cdict,256)

___
Now, you load the data and read out the desired variables from a NetCDF file from the ECMWF ERA-Interim data archive. You are loading a file with data from 1 January 2016.

In [ ]:
# Loading the ERA data.
nc_file = nc.Dataset("baroclinic.nc", "r")
lat = nc_file.variables["latitude"][:]
lon = nc_file.variables["longitude"][:]
p = nc_file.variables["level"][:]*100.
time = nc.num2date(nc_file.variables["time"][:], nc_file.variables["time"].units)

# We load 3D fields of the two horizontal wind components, the geopotential and the temperature.
u     = nc_file.variables["u"][:,:,:,:]
v     = nc_file.variables["v"][:,:,:,:]
omega = nc_file.variables["w"][:,:,:,:]
Phi   = nc_file.variables["z"][:,:,:,:]
T     = nc_file.variables["t"][:,:,:,:]
msl   = nc_file.variables["msl"][:,:,:] # Mean sea level pressure.

cp    = nc_file.variables["cp"][:,:,:]  # Convective precipitation.
lsp   = nc_file.variables["lsp"][:,:,:] # Large-scale precipitation.
pr = cp + lsp

nc_file.close() # The file access is closed as no further data is needed now.

___
# Plotting the temperature and geopotential on a map
As a starting point you find below the temperature and geopotential at 500 hPa.

In [ ]:
n = abs(p-50000).argmin()

my_projection = ccrs.PlateCarree(central_longitude=0)

def plot_map(nt=0):
    fig1 = plt.figure()
    ax1 = plt.subplot(111, projection=my_projection)

    #ax1.add_feature(cartopy.feature.COASTLINE, linewidth=0.8)
    ax1.coastlines('50m', linewidth=0.8)
    ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)

    ax1.set_xticks(np.arange(-180, 181, 30), crs=my_projection)
    ax1.set_yticks(np.arange(-90, 91, 30), crs=my_projection)
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax1.xaxis.set_major_formatter(lon_formatter)
    ax1.yaxis.set_major_formatter(lat_formatter)

    Tmin, Tmax = T[:,n,:,:].min(), T[:,n,:,:].max()
    Phi_range = np.arange(0, 90000, 1000)
    cb = ax1.pcolormesh(lon, lat, T[nt,n,:,:], cmap=my_cmap) # We plot a colormesh using the gist_ncar colormap.
    fig1.colorbar(cb) # We add a colorbar to show the values of temperature.
    cz = ax1.contour(lon, lat, Phi[nt,n,:,:], Phi_range, colors='w', linewidths=1.5) # We plot the geopotential in contours.
    ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
    #cs = ax1.contour(lon, lat, msl[nt,:,:], 15, colors='k', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.


    ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
    fig1.tight_layout() # With this function we make the figure fit as good as possible.
    
interact(plot_map, nt=(0, u.shape[0]-1, 1));

___
Below you find an example of how to construct a time series of a statistical quantity. You will adapt this example later to study the cyclone strength.

In [ ]:
pr_max = pr.max(axis=2).max(axis=1)

fig = plt.figure()
ax1 = plt.subplot(111)
ax1.plot(time, pr_max*1000, 'o-')
ax1.set_xlabel('time')
ax1.set_ylabel('mm')
fig.tight_layout()

Calculating gradients is slightly different compared to previous assignments, because you are dealing with a 4D array that has dimensions of `time`, `p`, `lat`, and `lon`, thus 4 axes. Below you can find an example how to calculate the geopotential $\Phi$ gradients.

In [ ]:
a_earth = 6.37e6

latrad = np.deg2rad(lat)
lonrad = np.deg2rad(lon)

# We initialize an array with the cosine of the latitude.
cos_lat = np.cos(latrad)

# Two arrays of zeros are initialized with the same shape as that of Z.
dPhidx = np.zeros(Phi.shape)
dPhidy = np.zeros(Phi.shape)

dPhidx[:,:,:,:] = (a_earth*cos_lat[np.newaxis,np.newaxis,:,np.newaxis])**(-1) \
                * np.gradient(Phi, axis=3) / np.gradient(lonrad[np.newaxis, np.newaxis, np.newaxis, :], axis=3)
dPhidy[:,:,:,:] = (a_earth)**(-1) \
                * np.gradient(Phi, axis=2) / np.gradient(latrad[np.newaxis,np.newaxis,:,np.newaxis], axis=2)

___
# Assignment
In this final assignment, you will study the evolution of the cyclone, and repeat techniques and concepts of previous weeks. If it helps to zoom into the system in some of the plots, feel free to do so.

## Time evolution of the cyclone through baroclinic instability
Use the example plot to follow the evolution of the midlatitude cyclone that forms east of Canada.
1. Describe the evolution of the geopotential and temperature of the cyclone at the 500 hPa from the first time step until the end. Can you observe the strenghtening and weakening of the cyclone?
2. The field `msl` contains the mean sea level pressure. Make a plot of the time evolution of the minimum value of mean sea level pressure over time. Adapt the $xy$-plot above.
3. Explain the figure. When does the minimum pressure occur? What happens faster, the strengthening or the weakening of the system?
___

## The connection between the upper atmosphere and the surface
1. Plot a map of the divergence at 300 hPa and add contour lines (solid lines) of the geopotential, and contour lines (dotted lines) of the mean sea level pressure. Do this 12 h before the minimum pressure occurs. 
2. Is the low pressure system at the surface at the expected location with respect to the 300 hPa geopotential structure?
___
## The evolution of temperature fronts and precipitation
1. Calculate the magnitude of the temperature gradient vector.
2. Plot a map of the magnitude of the temperature gradient vector at 850 hPa. Add again contour lines of the geopotential (solid lines) at 300 hPa, and of the mean sea level pressure (dotted lines). Do this 12 h before the moment of the minimum mean sea level pressure, at the moment of minimum pressure, and 12 h after the moment.
3. Do you observe the cold front and warm front? When does the occlusion occur?
___
## Precipitation in the cyclone.
1. Plot maps with precipitation `pr` and vertical motion `omega` for the same three moments in time as the previous question.
2. Where in the system does the most precipitation occur? What is the vertical velocity in those regions?

## Temperature in the top of the troposphere
1. Plot a map of temperature at 200 hPa at the moment of the minimum mean sea level pressure.
2. Explain the structure near the cyclone.
___
Work out the assignment below.

In [ ]:
p_min = msl.min(axis=-1).min(axis=-1)

fig = plt.figure()
ax1 = plt.subplot(111)
ax1.plot(time, p_min, 'o-')
fig.tight_layout()

Study the system at the early development: 4 January 2017, 00:00:00 UTC. Is the system developing at its logical place?

In [ ]:
nt = 4
n = abs(p-30000).argmin()

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.coastlines('50m', linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)
ax1.set_xticks(np.arange(-180, 181, 30), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 30), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

Tmin, Tmax = T[:,n,:,:].min(), T[:,n,:,:].max()
Phi_range = np.arange(0, 110000, 2000)
msl_range = np.arange(90000, 105000, 1000)
#cb = ax1.pcolormesh(lon, lat, T[nt,n,:,:], cmap=plt.cm.viridis) # We plot a colormesh using the gist_ncar colormap.
#fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

cz = ax1.contour(lon, lat, Phi[nt,n,:,:]/100, Phi_range/100, colors='k', linewidths=1.5) # We plot the geopotential in contours.
ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
cs = ax1.contour(lon, lat, msl[nt,:,:]/100, msl_range/100, colors='b', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.
ax1.clabel(cs, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.

ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
fig1.tight_layout() # With this function we make the figure fit as good as possible.

In [ ]:
Omega = 7.292e-5
a_earth = 6.37e6
g = 9.81
R = 287.

In [ ]:
a_earth = 6.37e6

latrad = np.deg2rad(lat)
lonrad = np.deg2rad(lon)

# We initialize an array with the cosine of the latitude.
cos_lat = np.cos(latrad)

# Two arrays of zeros are initialized with the same shape as that of Z.
dPhidx = np.zeros(Phi.shape)
dPhidy = np.zeros(Phi.shape)

dPhidx[:,:,:,:] = (a_earth*cos_lat[np.newaxis,np.newaxis,:,np.newaxis])**(-1) \
                * np.gradient(Phi, axis=3) / np.gradient(lonrad[np.newaxis, np.newaxis, np.newaxis, :], axis=3)
dPhidy[:,:,:,:] = (a_earth)**(-1) \
                * np.gradient(Phi, axis=2) / np.gradient(latrad[np.newaxis,np.newaxis,:,np.newaxis], axis=2)

dudx = (a_earth*cos_lat[np.newaxis,np.newaxis,:,np.newaxis])**(-1) \
     * np.gradient(u, axis=3) / np.gradient(lonrad[np.newaxis, np.newaxis, np.newaxis, :], axis=3)
dvdy = (a_earth)**(-1) \
     * np.gradient(v, axis=2) / np.gradient(latrad[np.newaxis,np.newaxis,:,np.newaxis], axis=2)
div = dudx + dvdy
    
f = 2.*Omega*np.sin(latrad)
ug = -(1./f[np.newaxis,np.newaxis,:,np.newaxis])*dPhidy
vg =  (1./f[np.newaxis,np.newaxis,:,np.newaxis])*dPhidx
Vg = (ug**2 + vg**2)**.5
V = (u**2 + v**2)**.5

In [ ]:
lon_min = -90
lon_max = -10
lat_min = 30
lat_max = 65

In [ ]:
# Divergence plot.
nt = 4
n = abs(p-30000).argmin()

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.coastlines('50m', linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)
ax1.set_xticks(np.arange(-180, 181, 30), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 30), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

div_min, div_max = -2e-4, 2e-4
Phi_range = np.arange(0, 110000, 2000)
msl_range = np.arange(90000, 105000, 1000)
cb = ax1.pcolormesh(lon, lat, div[nt,n,:,:], vmin=div_min, vmax=div_max, cmap=plt.cm.seismic)
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

cz = ax1.contour(lon, lat, Phi[nt,n,:,:]/100, Phi_range/100, colors='k', linewidths=1.5) # We plot the geopotential in contours.
ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
cs = ax1.contour(lon, lat, msl[nt,:,:]/100, msl_range/100, colors='b', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cs, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.

ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
fig1.tight_layout() # With this function we make the figure fit as good as possible.

In [ ]:
dTdx = (a_earth*cos_lat[np.newaxis,np.newaxis,:,np.newaxis])**(-1) \
     * np.gradient(T, axis=3) / np.gradient(lonrad[np.newaxis, np.newaxis, np.newaxis, :], axis=3)
dTdy = (a_earth)**(-1) \
     * np.gradient(T, axis=2) / np.gradient(latrad[np.newaxis,np.newaxis,:,np.newaxis], axis=2)
Tgrad = (dTdx**2 + dTdy**2)**.5

In [ ]:
# Omega plot.
nt = 6
n = abs(p-60000).argmin()
n300 = abs(p-30000).argmin()

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.coastlines('50m', linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)
ax1.set_xticks(np.arange(-180, 181, 30), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 30), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

Phi_range = np.arange(0, 110000, 2000)
msl_range = np.arange(90000, 105000, 1000)
cb = ax1.pcolormesh(lon, lat, omega[nt,n,:,:], cmap=plt.cm.seismic, vmin=-4, vmax=4)
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

cz = ax1.contour(lon, lat, Phi[nt,n300,:,:]/100, Phi_range/100, colors='k', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
cs = ax1.contour(lon, lat, msl[nt,:,:]/100, msl_range/100, colors='k', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cs, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
ax1.set_xlim(lon_min, lon_max)
ax1.set_ylim(lat_min, lat_max)
ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
fig1.tight_layout() # With this function we make the figure fit as good as possible.

In [ ]:
# Fronts plot.
nt = 4
n = abs(p-85000).argmin()
n300 = abs(p-30000).argmin()

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.coastlines('50m', linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)
ax1.set_xticks(np.arange(-180, 181, 30), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 30), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

Phi_range = np.arange(0, 110000, 2000)
msl_range = np.arange(90000, 105000, 1000)
cb = ax1.pcolormesh(lon, lat, Tgrad[nt,n,:,:], cmap=my_cmap, vmin=0, vmax=1.6e-4)
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

cz = ax1.contour(lon, lat, Phi[nt,n300,:,:]/100, Phi_range/100, colors='k', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
cs = ax1.contour(lon, lat, msl[nt,:,:]/100, msl_range/100, colors='k', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cs, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
ax1.set_xlim(lon_min, lon_max)
ax1.set_ylim(lat_min, lat_max)
ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
fig1.tight_layout() # With this function we make the figure fit as good as possible.

In [ ]:
# Precipitation plots.
nt = 4
n = abs(p-85000).argmin()
n300 = abs(p-30000).argmin()

# Convective precipitation.
fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.coastlines('50m', linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)
ax1.set_xticks(np.arange(-180, 181, 30), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 30), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

Phi_range = np.arange(0, 110000, 2000)
msl_range = np.arange(90000, 105000, 1000)
cb = ax1.pcolormesh(lon, lat, cp[nt,:,:]*1e3, cmap=my_cmap, vmin=0, vmax=4)
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

cz = ax1.contour(lon, lat, Phi[nt,n300,:,:]/100, Phi_range/100, colors='k', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
cs = ax1.contour(lon, lat, msl[nt,:,:]/100, msl_range/100, colors='k', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cs, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
#ax1.set_xlim(lon_min, lon_max)
#ax1.set_ylim(lat_min, lat_max)
ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
fig1.tight_layout() # With this function we make the figure fit as good as possible.

# Large-scale precipitation.
fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.coastlines('50m', linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)
ax1.set_xticks(np.arange(-180, 181, 30), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 30), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

Phi_range = np.arange(0, 110000, 2000)
msl_range = np.arange(90000, 105000, 1000)
cb = ax1.pcolormesh(lon, lat, lsp[nt,:,:]*1e3, cmap=my_cmap, vmin=0, vmax=4)
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

cz = ax1.contour(lon, lat, Phi[nt,n300,:,:]/100, Phi_range/100, colors='k', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
cs = ax1.contour(lon, lat, msl[nt,:,:]/100, msl_range/100, colors='k', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cs, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
#ax1.set_xlim(lon_min, lon_max)
#ax1.set_ylim(lat_min, lat_max)
ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
fig1.tight_layout() # With this function we make the figure fit as good as possible.

# Total precipitation.
pr = cp + lsp

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.coastlines('50m', linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)
ax1.set_xticks(np.arange(-180, 181, 30), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 30), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

Phi_range = np.arange(0, 110000, 2000)
msl_range = np.arange(90000, 105000, 1000)
cb = ax1.pcolormesh(lon, lat, pr[nt,:,:]*1e3, cmap=my_cmap, vmin=0, vmax=4)
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

cz = ax1.contour(lon, lat, Phi[nt,n300,:,:]/100, Phi_range/100, colors='k', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
cs = ax1.contour(lon, lat, msl[nt,:,:]/100, msl_range/100, colors='k', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cs, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
#ax1.set_xlim(lon_min, lon_max)
#ax1.set_ylim(lat_min, lat_max)
ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
fig1.tight_layout() # With this function we make the figure fit as good as possible.

In [ ]:
# Q-vector
dugdx = (a_earth*cos_lat[np.newaxis,np.newaxis,:,np.newaxis])**(-1) \
      * np.gradient(ug, axis=3) / np.gradient(lonrad[np.newaxis, np.newaxis, np.newaxis, :], axis=3)
dvgdx = (a_earth*cos_lat[np.newaxis,np.newaxis,:,np.newaxis])**(-1) \
      * np.gradient(vg, axis=3) / np.gradient(lonrad[np.newaxis, np.newaxis, np.newaxis, :], axis=3)
dugdy = (a_earth)**(-1) \
      * np.gradient(ug, axis=2) / np.gradient(latrad[np.newaxis,np.newaxis,:,np.newaxis], axis=2)
dvgdy = (a_earth)**(-1) \
      * np.gradient(vg, axis=2) / np.gradient(latrad[np.newaxis,np.newaxis,:,np.newaxis], axis=2)

Qi = -R/p[np.newaxis,:,np.newaxis,np.newaxis]*(dugdx*dTdx + dvgdx*dTdy)
Qj = -R/p[np.newaxis,:,np.newaxis,np.newaxis]*(dugdy*dTdx + dvgdy*dTdy)
dQidx = (a_earth*cos_lat[np.newaxis,np.newaxis,:,np.newaxis])**(-1) \
      * np.gradient(Qi, axis=3) / np.gradient(lonrad[np.newaxis, np.newaxis, np.newaxis, :], axis=3)
dQjdy = (a_earth)**(-1) \
      * np.gradient(Qj, axis=2) / np.gradient(latrad[np.newaxis,np.newaxis,:,np.newaxis], axis=2)
divQ = dQidx + dQjdy

In [ ]:
dvdx = (a_earth*cos_lat[np.newaxis,np.newaxis,:,np.newaxis])**(-1) \
     * np.gradient(v, axis=3) / np.gradient(lonrad[np.newaxis, np.newaxis, np.newaxis, :], axis=3)

dudy = (a_earth)**(-1) \
     * np.gradient(u, axis=2) / np.gradient(latrad[np.newaxis,np.newaxis,:,np.newaxis], axis=2)

zeta = dvdx - dudy

In [ ]:
nt = 4
n = abs(p-50000).argmin()

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.coastlines('50m', linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)
ax1.set_xticks(np.arange(-180, 181, 30), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 30), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

Phi_range = np.arange(0, 110000, 1000)
msl_range = np.arange(90000, 105000, 1000)
cb = ax1.pcolormesh(lon, lat, zeta[nt,n,:,:], cmap=plt.cm.seismic, vmin=-3e-4, vmax=3e-4)
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

cz = ax1.contour(lon, lat, Phi[nt,n,:,:]/100, Phi_range/100, colors='k', linewidths=1.5) # We plot the geopotential in contours.
ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
cs = ax1.contour(lon, lat, msl[nt,:,:]/100, msl_range/100, colors='k', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.
ax1.clabel(cs, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
#ax1.set_xlim(lon_min, lon_max)
#ax1.set_ylim(lat_min, lat_max)
ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
fig1.tight_layout() 

In [ ]:
nt = 6
n = abs(p-20000).argmin()

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.coastlines('50m', linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)
ax1.set_xticks(np.arange(-180, 181, 30), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 30), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

Phi_range = np.arange(0, 110000, 1500)
msl_range = np.arange(90000, 105000, 1000)
cb = ax1.pcolormesh(lon, lat, T[nt,n,:,:], cmap=my_cmap)
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.

cz = ax1.contour(lon, lat, Phi[nt,n300,:,:]/100, Phi_range/100, colors='w', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
fig1.tight_layout() 

In [ ]:
# Q-vector plot
#lon_min, lon_max, lat_min, lat_max = -90, -40, 30, 60
nt = 4
n = abs(p-85000).argmin()

fig1 = plt.figure()
ax1 = plt.subplot(111, projection=my_projection)

ax1.coastlines('50m', linewidth=0.8)
ax1.add_feature(cartopy.feature.BORDERS, linestyle='-', linewidth=.2)
ax1.set_xticks(np.arange(-180, 181, 10), crs=my_projection)
ax1.set_yticks(np.arange(-90, 91, 10), crs=my_projection)
lon_formatter = LongitudeFormatter(zero_direction_label=True)
lat_formatter = LatitudeFormatter()
ax1.xaxis.set_major_formatter(lon_formatter)
ax1.yaxis.set_major_formatter(lat_formatter)

Phi_range = np.arange(0, 110000, 2000)
msl_range = np.arange(90000, 105000, 1000)
cb = ax1.pcolormesh(lon, lat, divQ[nt,n,:,:], cmap=plt.cm.seismic, vmin=-1e-14, vmax=1.e-14)
fig1.colorbar(cb) # We add a colorbar to show the values of temperature.
nq = 2
lons, lats = np.meshgrid(lon, lat)
qu = ax1.quiver(lons[::nq,::nq], lats[::nq,::nq], Qi[nt,n,::nq,::nq], Qj[nt,n,::nq,::nq],\
                pivot='mid', width=1.2e-3, scale=5e-9)
cz = ax1.contour(lon, lat, Phi[nt,n,:,:]/100, Phi_range/100, colors='k', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cz, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
#cs = ax1.contour(lon, lat, msl[nt,:,:]/100, msl_range/100, colors='k', linestyles=':', linewidths=1.5) # We plot the geopotential in contours.
#ax1.clabel(cs, fmt='%1.0f', fontsize=10.) # We add labels to the contour lines.
ax1.set_xlim(lon_min, lon_max)
ax1.set_ylim(lat_min, lat_max)
ax1.set_title(r'T and $\Phi$ at p = {:.0f} hPa at {:}'.format(p[n]/100., time[nt])); # We add a title to the plot.
fig1.tight_layout() # With this function we make the figure fit as good as possible.

---